<a href="https://colab.research.google.com/github/100jy/voice_competition/blob/master/large_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글 드라이브 연동
# 클라우드 권한 획득
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
import keras
from tqdm import tqdm
from glob import glob
from scipy.io import wavfile
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, Callback
from tensorflow.keras.optimizers import *
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
import tensorflow_addons as tfa

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [2]:
def set_batch(size=1600,seed=100): 
  GCS_PATH = 'gs://data_bucket_9586/' 
  AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

  def read_train(example):
    feature = {'data': tf.io.VarLenFeature(dtype = tf.float32),
              'label': tf.io.VarLenFeature(dtype = tf.float32),
              }
    example = tf.io.parse_single_example(example, feature)

    data = tf.sparse.to_dense(example['data'])
    label = tf.sparse.to_dense(example['label'])

    data = tf.reshape(data,(299,299,1))
    label = tf.reshape(label,(30,1))

    return data, label

  filenames = tf.io.gfile.glob(GCS_PATH + 'train_resize_*.tfrec')[:200000]
  spit_len = int(len(filenames) * 0.8)

  np.random.seed(seed)
  np.random.shuffle(filenames)

  global train_dataset
  train_dataset = tf.data.TFRecordDataset(filenames[:spit_len],num_parallel_reads=AUTO)
  #batch_size = 3200
  train_dataset = train_dataset.map(read_train, num_parallel_calls=AUTO).batch(size).prefetch(1)

  global val_dataset
  val_dataset = tf.data.TFRecordDataset(filenames[spit_len:],num_parallel_reads=AUTO)
  #batch_size = 3200
  val_dataset = val_dataset.map(read_train, num_parallel_calls=AUTO).batch(size).prefetch(1)  

  for record in train_dataset.take(1):
    print(len(record))
    input_shape = (record[0].shape[1],record[0].shape[2],record[0].shape[3])
    output_shape = record[1].shape[1]

  print(input_shape)
  print(output_shape)

  # Detect hardware
  try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  except ValueError:
    tpu_resolver = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

  # Select appropriate distribution strategy
  if tpu_resolver:
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)

  elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])

  elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

  else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

  return strategy, train_dataset, val_dataset

In [3]:
def inception_model_2(input_shape):
  def inception_module(x, o_1=64,r_3 =64, o_3 =128,r_5=16,o_5=32,pool=32):
    #size_1 filter
    x_1 = Conv2D(o_1,1,padding='same',kernel_regularizer=l2(1e-5))(x)

    #size_1 + size_5 filter
    x_3 = Conv2D(r_5,1,padding='same',kernel_regularizer=l2(1e-5))(x)
    x_3 = Conv2D(o_5,5,padding='same',kernel_regularizer=l2(1e-5))(x_3)

    #pooling
    x_4 = MaxPooling2D(pool_size = (3,3),strides =1,padding='same')(x)
    x_4 = Conv2D(pool, 1, padding='same',kernel_regularizer=l2(1e-5))(x_4)

    return concatenate([x_1,x_3,x_4])

  #DeepCNN with Gelu
  def gelu(x):
    return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))


  inp = Input(input_shape) 

  x = Conv2D(64, (7, 7), strides = 2, padding = "same", kernel_initializer='he_normal')(inp)
  x = Activation(gelu)(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (1, 1), strides = 1, padding = "same", kernel_initializer='he_normal')(x)
  x = Activation(gelu)(x)
  x = Conv2D(192, (3, 3), strides = 1, padding = "same", kernel_initializer='he_normal')(x)
  x = Activation(gelu)(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)
  x = BatchNormalization()(x)

  x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=128, r_3=128, o_3=192, r_5=32, o_5=96, pool=64)
  x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=192, r_3=96, o_3=208, r_5=16, o_5=48, pool=64)
  x = BatchNormalization()(x)
  
  x = inception_module(x, o_1=160, r_3=112, o_3=224, r_5=24, o_5=64, pool=64)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=128, r_3=128, o_3=256, r_5=24, o_5=64, pool=64)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=112, r_3=144, o_3=288, r_5=32, o_5=64, pool=64)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)

  x = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(x)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=384, r_3=192, o_3=384, r_5=48, o_5=128, pool=128)
  

  x = AveragePooling2D(pool_size=(2, 2), strides=3)(x)
  x = Conv2D(128, (1, 1),padding = "same",kernel_regularizer=l2(1e-5))(x)
  x = Activation(gelu)(x)
  x = Flatten()(x)
  x = Dropout(0.8)(x)


  x = Dense(1024)(x)
  x = Activation(gelu)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.8)(x)


  output = Dense(30, activation = "softmax")(x)

  return Model(inp,output)


In [4]:
def inception_model_1(input_shape):
  def inception_module(x, o_1=64,r_3 =64, o_3 =128,r_5=16,o_5=32,pool=32):
    #size_1 filter
    x_1 = Conv2D(o_1,1,padding='same',kernel_regularizer=l2(1e-5))(x)

    #size_1 + size_5 filter
    x_3 = Conv2D(r_5,1,padding='same',kernel_regularizer=l2(1e-5))(x)
    x_3 = Conv2D(o_5,5,padding='same',kernel_regularizer=l2(1e-5))(x_3)

    #pooling
    x_4 = MaxPooling2D(pool_size = (3,3),strides =1,padding='same')(x)
    x_4 = Conv2D(pool, 1, padding='same',kernel_regularizer=l2(1e-5))(x_4)

    return concatenate([x_1,x_3,x_4])

  #DeepCNN with Gelu
  def gelu(x):
      return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))


  inp = Input(input_shape) 

  x = Conv2D(64, (7, 7), strides = 2, padding = "same",kernel_regularizer=l2(1e-5))(inp)
  x = Activation(gelu)(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)
  x = Conv2D(64, (1, 1), strides = 1, padding = "same",kernel_regularizer=l2(1e-5))(x)
  x = Activation(gelu)(x)
  x = Conv2D(192, (3, 3), strides = 1, padding = "same",kernel_regularizer=l2(1e-5))(x)
  x = Activation(gelu)(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)

  x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)
  x = inception_module(x, o_1=128, r_3=128, o_3=192, r_5=32, o_5=96, pool=64)
  x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
  x = inception_module(x, o_1=192, r_3=96, o_3=208, r_5=16, o_5=48, pool=64)
  
  x = inception_module(x, o_1=160, r_3=112, o_3=224, r_5=24, o_5=64, pool=64)
  x = inception_module(x, o_1=128, r_3=128, o_3=256, r_5=24, o_5=64, pool=64)
  x = inception_module(x, o_1=112, r_3=144, o_3=288, r_5=32, o_5=64, pool=64)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)

  x = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(x)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
  x = inception_module(x, o_1=384, r_3=192, o_3=384, r_5=48, o_5=128, pool=128)
  

  x = AveragePooling2D(pool_size=(2, 2), strides=3)(x)
  x = Conv2D(128, (1, 1),padding = "same",kernel_regularizer=l2(1e-5))(x)
  x = Activation(gelu)(x)
  x = Flatten()(x)
  #x = Dropout(0.8)(x)

  '''
  x = Dense(1024)(x)
  x = Activation(gelu)(x)
  x = Dropout(0.8)(x)
  '''

  output = Dense(30, activation = "softmax")(x)

  return Model(inp,output)


In [5]:
import tensorflow.keras.backend as K

def conv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu'):
    x = Conv2D(filters, kernel_size, kernel_initializer='he_normal', padding=padding, strides=strides)(x)
    x = BatchNormalization()(x)
    if activation:
        x = Activation(activation)(x)
    
    return x


def SE_block(input_tensor, reduction_ratio=16):
    ch_input = K.int_shape(input_tensor)[-1]
    ch_reduced = ch_input//reduction_ratio
    
    # Squeeze
    x = GlobalAveragePooling2D()(input_tensor) # Eqn.2
    
    # Excitation
    x = Dense(ch_reduced, kernel_initializer='he_normal', activation='relu', use_bias=False)(x) # Eqn.3
    x = Dense(ch_input, kernel_initializer='he_normal', activation='sigmoid', use_bias=False)(x) # Eqn.3
    
    x = Reshape( (1, 1, ch_input) )(x)
    x = Multiply()([input_tensor, x]) # Eqn.4
    
    return x
   

def SE_residual_block(input_tensor, filter_sizes, strides=1, reduction_ratio=16):
    filter_1, filter_2, filter_3 = filter_sizes
    
    x = conv2d_bn(input_tensor, filter_1, (1, 1), strides=strides)
    x = conv2d_bn(x, filter_2, (3, 3))
    x = conv2d_bn(x, filter_3, (1, 1), activation=None)
    
    x = SE_block(x, reduction_ratio)
    
    projected_input = conv2d_bn(input_tensor, filter_3, (1, 1), strides=strides, activation=None) if K.int_shape(input_tensor)[-1] != filter_3 else input_tensor
    shortcut = Add()([projected_input, x])
    shortcut = Activation(activation='relu')(shortcut)
    
    return shortcut
 

def stage_block(input_tensor, filter_sizes, blocks, reduction_ratio=16, stage=''):
    strides = 2 if stage != '2' else 1
    
    x = SE_residual_block(input_tensor, filter_sizes, strides, reduction_ratio) # projection layer

    for i in range(blocks-1):
        x = SE_residual_block(x, filter_sizes, reduction_ratio=reduction_ratio)
    
    return x
    

def SE_ResNet50(input_shape, classes=30):
    model_input = Input(input_shape)
    stage_1 = conv2d_bn(model_input, 64, (7, 7), strides=2, padding='same') # (112, 112, 64)
    stage_1 = MaxPooling2D((3, 3), strides=2, padding='same')(stage_1) # (56, 56, 64)
    
    stage_2 = stage_block(stage_1, [64, 64, 256], 3, reduction_ratio=16, stage='2')
    stage_3 = stage_block(stage_2, [128, 128, 512], 4, reduction_ratio=16, stage='3') # (28, 28, 512)
    stage_4 = stage_block(stage_3, [256, 256, 1024], 6, reduction_ratio=16, stage='4') # (14, 14, 1024)
    stage_5 = stage_block(stage_4, [512, 512, 2048], 3, reduction_ratio=16, stage='5') # (7, 7, 2048)

    gap = GlobalAveragePooling2D()(stage_5)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(gap) # 'softmax'
    
    model = Model(inputs=model_input, outputs=model_output, name='SE-ResNet50')
        
    return model

In [6]:
def fit_model(id, n_epoch,strategy, train_dataset, val_dataset, road_weight =False,model_name ='m1',optimizer_set='radam'):
  model_path = 'drive/My Drive/data/model_high_resol_{}/'.format(id)
 
  if not os.path.exists(model_path):
    os.mkdir(model_path)
   
      
  # validattion 기준 모델 갱신
  model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
  checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

  if optimizer_set == 'radam':
    radam = tfa.optimizers.RectifiedAdam()
    optimizer  = tfa.optimizers.Lookahead(radam, sync_period=3, slow_step_size=0.5)

  elif optimizer_set == 'lamb':
    ramb = tfa.optimizers.LAMB()
    optimizer  = tfa.optimizers.Lookahead(ramb, sync_period=6, slow_step_size=0.5)


  with strategy.scope():
    if model_name == 'inception_1':
      model = inception_model_1((299,299,1))
    elif model_name == 'inception_2':
      model = inception_model_2((299,299,1))
    elif model_name == 'SE_ResNet':
      model = SE_ResNet50((299,299,1),30)
    model.compile(loss=tf.keras.losses.KLDivergence(), optimizer = optimizer)
    if road_weight:
      model.load_weights(glob('drive/My Drive/data/model_high_resol_{}/*.hdf5'.format(id))[-1])
    history = model.fit(train_dataset, epochs = n_epoch, validation_data=val_dataset, callbacks = [checkpoint])

In [12]:
def fit_k_models(k = 5, model_name='SE_ResNet',batch_size = 64):
  for i in range(1,k+1):
    strategy, train_dataset, val_dataset = set_batch(batch_size,i)
    fit_model(id=i, n_epoch=6, strategy=strategy, train_dataset=train_dataset, val_dataset=val_dataset, road_weight=False ,model_name=model_name)

def load_test_set():
  GCS_PATH = 'gs://data_bucket_9586/' 
  AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

  def read_test(example):
    features = {'data': tf.io.VarLenFeature(dtype = tf.float32)}
    example = tf.io.parse_single_example(example, features)

    data = tf.sparse.to_dense(example['data'])

    data = tf.reshape(data,(299,299,1))
    return data
    
  filenames = tf.io.gfile.glob(GCS_PATH + 'test_*.tfrec')
  test_dataset = tf.data.TFRecordDataset(filenames)
  test_dataset = test_dataset.map(read_test).batch(128).prefetch(1)

  return test_dataset

def make_result(model_num):
  # 가장 좋은 모델의 weight를 불러옵니다.
  input_shape = (299,299,1)
  model = SE_ResNet50((299,299,1),30)
  weight_file = glob('drive/My Drive/data/model_high_resol_{}/*.hdf5'.format(model_num))[-1]
  print(weight_file)
  model.load_weights(weight_file)

  # 예측 수행
  y_pred = model.predict(test_dataset)

  return y_pred

def k_fold(k, model_name='SE_ResNet',batch_size = 64):
  #data load
  test_dataset = load_test_set()

  #model_fit
  fit_k_models(k = 5, model_name=model_name,batch_size = batch_size)

  #make_result
  submission = pd.read_csv('drive/My Drive/data/submission.csv', index_col=0)
  suma = np.zeros_like(submission)
  for i in range(1,k):
    suma += make_result(i)
  submission.loc[:, :] = suma / 5
  submission.to_csv('drive/My Drive/data/submission.csv')

In [ ]:
k_fold(5, model_name='SE_ResNet',batch_size = 64)

# 8/4
--------------------------------------

**모델** : SE_ResNet50(5-folds,6 epochs)
  
**데이터**: x_trian_mel-spectogram(299*299)
  
**score** : **0.72265 (15위)**

**메모** : shifted aug not good, k-folds good, 에폭 더 주면 많이 좋을 듯..

**피드백** : 12에폭으로 결과 확인

--------------------------------------